# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
url = "https://books.toscrape.com/"

response = requests.get(url)
response

<Response [200]>

In [6]:
print(response.content)

b'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:29" />\n        <meta name="description" content="" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shortcut icon" hre

In [7]:
type(response.content)

bytes

In [ ]:
# Map the star-rating CSS class to numbers
RATING_MAP = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}

def get_soup(url):
    """Fetch URL and return BeautifulSoup."""
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")

def absolutize(current_url, href):
    """
    Build an absolute URL using only basic string operations.
    """
    if href.startswith("http://") or href.startswith("https://"):
        return href
    if href.startswith("/"):
        return BASE_URL.rstrip("/") + href
    if href.startswith("catalogue/"):
        return BASE_URL + href
    
    base_dir = current_url.rsplit("/", 1)[0] + "/"
    return base_dir + href

def clean_price(text):
    """
    Turn '£51.77', 'Â£51.77', or '£ 1,234.50' into a float using only basic string ops.
    """
    
    ascii_text = text.encode("ascii", "ignore").decode()
    ascii_text = ascii_text.replace("£", "").replace(",", "").strip()
    
    
    keep = "".join(ch for ch in ascii_text if (ch.isdigit() or ch == "."))
    return float(keep) if keep else 0.0

def parse_book_card(article_tag, page_url):
    """
    Parse one book card from a listing page.
    Returns basic info + absolute detail URL.
    """
    a_tag = article_tag.find("h3").find("a")
    title = a_tag["title"].strip()
    rel_link = a_tag["href"]
    detail_url = absolutize(page_url, rel_link)

    price_text = article_tag.select_one(".price_color").get_text(strip=True)
    price = clean_price(price_text)

    rating_tag = article_tag.find("p", class_="star-rating")
    rating = None
    if rating_tag and rating_tag.has_attr("class"):
        classes = rating_tag["class"]  
        if len(classes) >= 2:
            rating = RATING_MAP.get(classes[1], None)

    availability = article_tag.select_one(".instock.availability").get_text(strip=True)

    return {
        "Title": title,
        "Price (£)": price,
        "Rating": rating,
        "Availability": availability,
        "DetailURL": detail_url,
    }

def parse_book_detail(detail_url):
    """
    Visit the detail page for UPC, Description, and Genre.
    """
    soup = get_soup(detail_url)

    
    upc = None
    table = soup.find("table", class_="table table-striped")
    if table:
        for tr in table.find_all("tr"):
            th = tr.find("th").get_text(strip=True)
            td = tr.find("td").get_text(strip=True)
            if th.lower() == "upc":
                upc = td
                break

    
    description = None
    desc_div = soup.find("div", id="product_description")
    if desc_div:
        p = desc_div.find_next_sibling("p")
        if p:
            description = p.get_text(strip=True)

    
    genre = None
    crumbs = soup.select("ul.breadcrumb li a")
    if len(crumbs) >= 3:
        genre = crumbs[2].get_text(strip=True)

    return {"UPC": upc, "Description": description, "Genre": genre}

def iter_listing_pages():
    """
    Yield (soup, url) for all catalogue pages, following the 'next' link.
    """
    current_url = BASE_URL
    while True:
        soup = get_soup(current_url)
        yield soup, current_url

        next_link = soup.select_one("li.next > a")
        if not next_link:
            break
        href = next_link.get("href", "")
        current_url = absolutize(current_url, href)

def scrape_books(min_rating, max_price):
    """
    Scrape Books to Scrape and return a DataFrame with:
      UPC, Title, Price (£), Rating, Genre, Availability, Description
    Keep only books with Rating >= min_rating and Price <= max_price.
    """
    rows = []

    for soup, page_url in iter_listing_pages():
        for art in soup.select("article.product_pod"):
            basic = parse_book_card(art, page_url)

            
            if basic["Rating"] is None or basic["Rating"] < min_rating:
                continue
            if basic["Price (£)"] > max_price:
                continue

            detail = parse_book_detail(basic["DetailURL"])
            rows.append({
                "UPC": detail["UPC"],
                "Title": basic["Title"],
                "Price (£)": basic["Price (£)"],
                "Rating": basic["Rating"],
                "Genre": detail["Genre"],
                "Availability": basic["Availability"],
                "Description": detail["Description"],
            })

    return pd.DataFrame(rows, columns=[
        "UPC", "Title", "Price (£)", "Rating", "Genre", "Availability", "Description"
    ])



In [9]:
df_books = scrape_books(min_rating=4.0, max_price=20.0)
print("Rows scraped:", len(df_books))
df_books.head(10)

Rows scraped: 75


,UPC,Title,Price (£),Rating,Genre,Availability,Description
0,ce6396b0f23f6ecc,Set Me Free,17.46,5,Young Adult,In stock,Aaron Ledbetterâs future had been planned ou...
1,6258a1f6a6dcfe50,The Four Agreements: A Practical Guide to Pers...,17.66,5,Spirituality,In stock,"In The Four Agreements, don Miguel Ruiz reveal..."
2,6be3beb0793a53e7,Sophie's World,15.94,5,Philosophy,In stock,A page-turning novel that is also an explorati...
3,657fe5ead67a7767,Untitled Collection: Sabbath Poems 2014,14.27,4,Poetry,In stock,"More than thirty-five years ago, when the weat..."
4,51653ef291ab7ddc,This One Summer,19.49,4,Sequential Art,In stock,"Every summer, Rose goes with her mom and dad t..."
5,709822d0b5bcb7f4,Thirst,17.27,5,Fiction,In stock,"On a searing summer Friday, Eddie Chapman has ..."
6,0fa6dceead7ce47a,"Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Pr...",13.61,5,Sequential Art,In stock,THE LONG-AWAITED STORY OF FANGIRLS TAKING ON T...
7,0e691eda369f4e09,Princess Between Worlds (Wide-Awake Princess #5),13.34,5,Fantasy,In stock,Just as Annie and Liam are busy making plans t...
8,889139b8e9c4cb36,"Outcast, Vol. 1: A Darkness Surrounds Him (Out...",15.44,4,Sequential Art,In stock,NEW HORROR SERIES FROM THE WALKING DEAD CREATO...
9,72f9d5be3472d34e,Mama Tried: Traditional Italian Cooking for th...,14.02,4,Food and Drink,In stock,Cecilia Granata grew up cooking with her famil...
